NHS England
# Hospital Admited Patient Care Activity

Data Source: [NHS Digital](https://digital.nhs.uk/data-and-information/publications/statistical/hospital-admitted-patient-care-activity)

This notebook presented a collection of analyses based on the publicly available data about hospital admitted patient care activity in England.

%md
## 1. Prepare Data

- Download patient care diagnosis data (.xlsx) from NHS England
- Upload the diagnosis data to Worksapce

**Path:** /Workspace/Users/_username_/Data/NHS England/Admitted Patient Care Activity/Diagnosis

## 2. Install dependencies

In [0]:
pip install openpyxl

In [0]:
%pip install pandas

## 3 - Load patient care diagnosis data

### 3.1 List patient care diagnosis data only

In [0]:
import pandas as pd

year = 24
path = f'/Workspace/Users/gaogang@gmail.com/Files/NHS England/Admitted Patient Care Activity/Diagnosis/hosp-epis-stat-admi-diag-20{year}-{year+1}-tab.xlsx'
sheet = 'Primary Diagnosis 3 Character '

df = pd.read_excel(path, sheet_name=sheet,usecols="A:B, H:P", names=["Code", "Primary Diagnosis", "Finished Consultant Episodes","Finished Admission Episodes","Male", "Female", "Gender Unknown", "Emergency", "Waiting List", "Planned", "Other"], skiprows=12)
df["Year"] = 2000 + year + 1

df = df[df["Primary Diagnosis"].notnull()]
# Convert columns to numeric, coercing errors to NaN
for col in [
    "Finished Consultant Episodes",
    "Finished Admission Episodes",
    "Male",
    "Female",
    "Gender Unknown", 
    "Emergency", 
    "Waiting List", 
    "Planned", 
    "Other"
]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

display(df)
display(df.dtypes)

### 3.2 Save patient care diagnosis data to unity catalogue

In [0]:
from pyspark.sql.types import IntegerType

spark_df = spark.createDataFrame(df)
spark_df = spark_df.withColumn("Year", spark_df["Year"].cast(IntegerType()))
spark_df.write.format("iceberg").mode("append").saveAsTable("`depq-nhs`.patient_care.diagnosis_lv2")

In [0]:
from delta.tables import DeltaTable

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Merge (upsert) into the Iceberg table using Code and Year as unique identifiers
spark_df.createOrReplaceTempView("tmp_diagnosis_update")

merge_sql = """
MERGE INTO `depq-nhs`.patient_care.diagnosis_lv2 AS target
USING tmp_diagnosis_update AS source
ON target.Code = source.Code AND target.Year = source.Year
WHEN MATCHED THEN UPDATE SET *
"""

spark.sql(merge_sql)

## 4. Simple analytics

### Display Year 2025 Data

In [0]:
df_2025 = spark.sql("""
SELECT *
FROM `depq-nhs`.patient_care.diagnosis_lv2
WHERE Year = 2025
""")
display(df_2025)

### Top 5 most common cancers

In [0]:
year = 2025

query = f"""
SELECT *
FROM `depq-nhs`.patient_care.diagnosis_lv2
WHERE ((Code >= 'C00' AND Code <= 'C97') OR (Code >= 'D00' AND Code <= 'D09'))
   AND Year = {year}
ORDER BY `Finished Admission Episodes` DESC
LIMIT 5
"""

df_cancer = spark.sql(query)
display(df_cancer)

## 5. Time series analytics

### Yearly Hospital Admissions for the Five Most Common Cancers

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

query = """
SELECT Year, Code, `Primary Diagnosis`, `Finished Admission Episodes`
FROM `depq-nhs`.patient_care.diagnosis_lv2
WHERE Year BETWEEN 2016 AND 2025
  AND Code IN ('C50', 'C90', 'C44', 'C18', 'C34')
ORDER BY Year, Code
"""

df_plot = spark.sql(query).toPandas()

plt.figure(figsize=(10,6))
for code in ['C50', 'C90', 'C44', 'C18', 'C34']:
    data = df_plot[df_plot['Code'] == code]
    if not data.empty:
        label = data['Primary Diagnosis'].iloc[0]
        plt.plot(data['Year'], data['Finished Admission Episodes'], marker='o', label=label)

plt.xlabel('Year')
plt.ylabel('Finished Admission Episodes')
plt.title('Finished Admission Episodes for Selected Diagnoses (2016-2025)')
plt.legend(title='Primary Diagnosis')
plt.grid(True)
plt.tight_layout()
plt.show()

### Prediction

#### Pre-requisites 

##### Prophet

In [0]:
pip install prophet

### Predict Number of Admission Episodes for Breast Cancer (2026-2030)

In [0]:
from prophet import Prophet
import pandas as pd

code = 'C50'
between = 2016
to = 2025

model_flexibility = 0.05

query = f"""
SELECT Year, `Finished Admission Episodes`
FROM `depq-nhs`.patient_care.diagnosis_lv2
WHERE `Code` = '{code}' AND Year BETWEEN {between} AND {to}
ORDER BY Year
"""

df_cancer = spark.sql(query).toPandas()
df_cancer = df_cancer.rename(columns={"Year": "ds", "Finished Admission Episodes": "y"})
df_cancer.loc[df_cancer["ds"].isin([2021]), "y"] = None
df_cancer["ds"] = pd.to_datetime(df_cancer["ds"], format='%Y')

display(df_cancer)

model = Prophet(changepoint_prior_scale=model_flexibility)
model.fit(df_cancer)

future = model.make_future_dataframe(periods=5, freq='Y')
forecast = model.predict(future)

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

fig1 = model.plot(forecast)
fig1.gca().set_xlabel('Year')
fig1.gca().set_ylabel('Finished Admission Episodes (Breast Cancer)')

display(forecast)